In [33]:
import pandas as pd
import datetime
import openpyxl
import xlsxwriter

from openpyxl.utils.dataframe import dataframe_to_rows

IRR Calculation Algo

In [10]:
def irr(dates,cashflows):
    diff = []
    for i in dates:
        if dates[0] == i:
            diff.append(0)
        else:
            diff.append((i-dates[0]).days)
    r = 0
    eq = 0
    eq_prev= 5
    ticker = 0
    trend = None
    step = 0.0001
    while True:
        for x in range(len(dates)):
            # print((cashflows[x]/((1+r)**(diff[x]/365))))
            eq = eq + (cashflows[x]/((1+r)**(diff[x]/365)))
            # print(eq)
        # print(eq,r,trend)
        if ticker == 0:
            eq_prev = eq
            eq = 0
            ticker += 1
            r = r + step
        elif ticker == 1:
            ticker += 1
            if eq < eq_prev and eq>0:
                trend = True
            elif eq > eq_prev and eq<0:
                trend = True
            else:
                trend = False
            eq_prev = eq
            eq = 0
        elif trend:
            ticker +=1
            if eq_prev >0:
                if eq < 0.01:
                    break
            elif eq_prev<0:
                if eq_prev > -0.01:
                    break
            r = r + step
            
            eq_prev = eq
            eq = 0
        else:
            if eq_prev >0:
                if eq < 0.01:
                    break
            elif eq_prev<0:
                if eq > -0.01:
                    break
            ticker +=1
            r = r - step
            
            eq_prev = eq
            eq = 0
        # eq = 0
    return r

XIRR from Stack Exchange

In [11]:
def xirr(dates,cashflows):
    transactions = []
    for i in range(len(dates)):
        transactions.append((dates[i],cashflows[i]))
    years = [(ta[0] - transactions[0][0]).days / 365.0 for ta in transactions]
    residual = 1
    step = 0.05
    guess = 0.05
    epsilon = 0.0001
    limit = 10000
    while abs(residual) > epsilon and limit > 0:
        limit -= 1
        residual = 0.0
        for i, ta in enumerate(transactions):
            residual += ta[1] / pow(guess, years[i])
        if abs(residual) > epsilon:
            if residual > 0:
                guess += step
            else:
                guess -= step
                step /= 2.0
    return guess-1



Normal SIP

Normal SIP.csv Import and gettind dates and cashflow for IRR

In [12]:
normalSIP = pd.read_csv('SIP Details/Normal SIP.csv')
normalSIP.drop(normalSIP.columns[[0]],axis=1,inplace=True)
d = []
cf = []
for i in normalSIP['Date']:
    if i == list(normalSIP['Date'])[-1]:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
    else:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
for x in normalSIP['Investment Amount']:
    if x == list(normalSIP['Investment Amount'])[-1]:
        cf.append(-x)
        qtytotal = normalSIP['Qty'].sum()
        ltp = list(normalSIP['Buy Price'])[-1]
        cf.append(qtytotal*ltp)
    else:
        cf.append(-x)

print(irr(d,cf)*100,'%')


print(xirr(d,cf))
# print(d)
# for i in d:
#     if d[0] == i:
#         diff.append(0)
#     else:

# print(type((d[1]-d[0]).days))
# print((cf))

14.350000000000051 %
0.14344816599041255


In [20]:
def returnAnalysis(filename):
    '''This Function takes in the File and gives out IRR for 1,3,5,7,10,15 and total timeframe'''

    '''Import Data'''
    normalSIP = pd.read_csv(f'SIP Details/{filename}.csv')
    normalSIP.drop(normalSIP.columns[[0]],axis=1,inplace=True)
    '''One Year Analysis'''
    l={}
    for i in list(normalSIP['Date']):
        # z = list(normalSIP['Date'].loc[lambda x:x==i].index)
        if (datetime.datetime.strptime(i,'%b %d, %Y').year) in l.keys():
            l[datetime.datetime.strptime(i,'%b %d, %Y').year].append(i)
        else:
            l[datetime.datetime.strptime(i,'%b %d, %Y').year] = [i]
    my_columns = ['Year','XIRR']
    normalSIP_oneyear = pd.DataFrame(columns=my_columns)
    for g in l.keys():
        d=[]
        cf = []
        qty = 0
        if g == 2022:
            continue
        for y in l[g]:
            z = list(normalSIP['Date'].loc[lambda x:x==y].index)
            if y == l[g][-1]:
                cf.append(-normalSIP.loc[z[0],'Investment Amount'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
                qty += normalSIP.loc[z[0],'Qty']
                cf.append(qty*normalSIP.loc[z[0],'Buy Price'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            else:
                cf.append(-normalSIP.loc[z[0],'Investment Amount'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
                qty += normalSIP.loc[z[0],'Qty']
        # print(g,irr(d,cf))
        normalSIP_oneyear = normalSIP_oneyear.append(pd.Series([str(g),irr(d,cf)],index=my_columns),ignore_index=True)

    normalSIP_oneyear.to_csv(f'Return Analysis/{filename}/One Year.csv')

    '''Three Year Analysis'''
    l={}
    for i in list(normalSIP['Date']):
        # z = list(normalSIP['Date'].loc[lambda x:x==i].index)
        if (datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
        datetime.datetime.strptime(i,'%b %d, %Y').year+2) in l.keys():
            # l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
            # datetime.datetime.strptime(i,'%b %d, %Y').year+2].append(None)
            continue
        else:
            l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
            datetime.datetime.strptime(i,'%b %d, %Y').year+2] = []
    # print(l)
    for z in l.keys():
        for date in list(normalSIP['Date']):
            if datetime.datetime.strptime(date,'%b %d, %Y').year in z:
                l[z].append(date)
            continue

    # print(l)

    my_columns = ['Year','XIRR']
    normalSIP_threeyear = pd.DataFrame(columns=my_columns)
    for g in l.keys():
        # print(l[g])
        d=[]
        cf = []
        qty = 0
        if 2022 in g:
            continue
        for y in l[g]:
            z = list(normalSIP['Date'].loc[lambda x:x==y].index)
            if y == l[g][-1]:
                cf.append(-normalSIP.loc[z[0],'Investment Amount'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
                qty += normalSIP.loc[z[0],'Qty']
                cf.append(qty*normalSIP.loc[z[0],'Buy Price'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            else:
                cf.append(-normalSIP.loc[z[0],'Investment Amount'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
                qty += normalSIP.loc[z[0],'Qty']
            
        # print(g,irr(d,cf))
        # print(d,cf)
        normalSIP_threeyear = normalSIP_threeyear.append(pd.Series([(f'{g[0]} - {g[-1]}'),irr(d,cf)],index=my_columns),ignore_index=True)

    normalSIP_threeyear.to_csv(f'Return Analysis/{filename}/Three Year.csv')


    '''Five Year Analysis'''
    l={}
    for i in list(normalSIP['Date']):
        # z = list(normalSIP['Date'].loc[lambda x:x==i].index)
        if (datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
        datetime.datetime.strptime(i,'%b %d, %Y').year+2,datetime.datetime.strptime(i,'%b %d, %Y').year+3,
        datetime.datetime.strptime(i,'%b %d, %Y').year+4) in l.keys():
            # l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
            # datetime.datetime.strptime(i,'%b %d, %Y').year+2,datetime.datetime.strptime(i,'%b %d, %Y').year+3,datetime.datetime.strptime(i,'%b %d, %Y').year+4].append(None)
            continue
        else:
            l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
            datetime.datetime.strptime(i,'%b %d, %Y').year+2,datetime.datetime.strptime(i,'%b %d, %Y').year+3,
            datetime.datetime.strptime(i,'%b %d, %Y').year+4] = []
    # print(l)
    for z in l.keys():
        for date in list(normalSIP['Date']):
            if datetime.datetime.strptime(date,'%b %d, %Y').year in z:
                l[z].append(date)
            continue

    # print(l)

    my_columns = ['Year','XIRR']
    normalSIP_fiveyear = pd.DataFrame(columns=my_columns)
    for g in l.keys():
        # print(l[g])
        d=[]
        cf = []
        qty = 0
        if 2022 in g:
            continue
        for y in l[g]:
            z = list(normalSIP['Date'].loc[lambda x:x==y].index)
            if y == l[g][-1]:
                cf.append(-normalSIP.loc[z[0],'Investment Amount'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
                qty += normalSIP.loc[z[0],'Qty']
                cf.append(qty*normalSIP.loc[z[0],'Buy Price'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            else:
                cf.append(-normalSIP.loc[z[0],'Investment Amount'])
                d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
                qty += normalSIP.loc[z[0],'Qty']
            
        # print(g,irr(d,cf))
        # print(d,cf)
        normalSIP_fiveyear = normalSIP_fiveyear.append(pd.Series([(f'{g[0]} - {g[-1]}'),irr(d,cf)],index=my_columns),ignore_index=True)

    normalSIP_fiveyear.to_csv(f'Return Analysis/{filename}/Five Year.csv')

                

            

            

        


            

        
# print(l)


In [22]:
returnAnalysis('Normal SIP')

1 Year Analysis

In [36]:
l={}
for i in list(normalSIP['Date']):
    # z = list(normalSIP['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%b %d, %Y').year) in l.keys():
        l[datetime.datetime.strptime(i,'%b %d, %Y').year].append(i)
    else:
        l[datetime.datetime.strptime(i,'%b %d, %Y').year] = [i]
my_columns = ['Year','XIRR']
normalSIP_oneyear = pd.DataFrame(columns=my_columns)
for g in l.keys():
    d=[]
    cf = []
    qty = 0
    if g == 2022:
        continue
    for y in l[g]:
        z = list(normalSIP['Date'].loc[lambda x:x==y].index)
        if y == l[g][-1]:
            cf.append(-normalSIP.loc[z[0],'Investment Amount'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            qty += normalSIP.loc[z[0],'Qty']
            cf.append(qty*normalSIP.loc[z[0],'Buy Price'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
        else:
            cf.append(-normalSIP.loc[z[0],'Investment Amount'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            qty += normalSIP.loc[z[0],'Qty']
    # print(g,irr(d,cf))
    normalSIP_oneyear = normalSIP_oneyear.append(pd.Series([str(g),irr(d,cf)],index=my_columns),ignore_index=True)

normalSIP_oneyear
rawb = openpyxl.load_workbook('SIP Analysis.xlsx')
ns = rawb['Normal SIP']
for i in range(len(list(normalSIP_oneyear['Year']))):
    if i == 0:
        ns['L3'] = 'Year'
        ns['M3'] = 'XIRR'
        ns[f'L{i+4}'] = list(normalSIP_oneyear['Year'])[i]
        ns[f'M{i+4}'] = list(normalSIP_oneyear['XIRR'])[i]
    else:
        ns[f'L{i+4}'] = list(normalSIP_oneyear['Year'])[i]
        ns[f'M{i+4}'] = list(normalSIP_oneyear['XIRR'])[i]
            
rawb.save('SIP Analysis.xlsx')
        
# print(l)


3 Year Analysis

In [19]:
l={}
for i in list(normalSIP['Date']):
    # z = list(normalSIP['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
    datetime.datetime.strptime(i,'%b %d, %Y').year+2) in l.keys():
        # l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
        # datetime.datetime.strptime(i,'%b %d, %Y').year+2].append(None)
        continue
    else:
        l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
        datetime.datetime.strptime(i,'%b %d, %Y').year+2] = []
# print(l)
for z in l.keys():
    for date in list(normalSIP['Date']):
        if datetime.datetime.strptime(date,'%b %d, %Y').year in z:
            l[z].append(date)
        continue

# print(l)

my_columns = ['Year','XIRR']
normalSIP_threeyear = pd.DataFrame(columns=my_columns)
for g in l.keys():
    # print(l[g])
    d=[]
    cf = []
    qty = 0
    if 2022 in g:
        continue
    for y in l[g]:
        z = list(normalSIP['Date'].loc[lambda x:x==y].index)
        if y == l[g][-1]:
            cf.append(-normalSIP.loc[z[0],'Investment Amount'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            qty += normalSIP.loc[z[0],'Qty']
            cf.append(qty*normalSIP.loc[z[0],'Buy Price'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
        else:
            cf.append(-normalSIP.loc[z[0],'Investment Amount'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            qty += normalSIP.loc[z[0],'Qty']
        
    # print(g,irr(d,cf))
    # print(d,cf)
    normalSIP_threeyear = normalSIP_threeyear.append(pd.Series([(f'{g[0]} - {g[-1]}'),irr(d,cf)],index=my_columns),ignore_index=True)

normalSIP_threeyear

            

        


,Year,XIRR
0,2002 - 2004,0.3337
1,2003 - 2005,0.3981
2,2004 - 2006,0.4423
3,2005 - 2007,0.4949
4,2006 - 2008,-0.2435
5,2007 - 2009,0.1479
6,2008 - 2010,0.2211
7,2009 - 2011,0.0340
8,2010 - 2012,0.0603
9,2011 - 2013,0.0837


5 Year Analysis

In [15]:
l={}
for i in list(normalSIP['Date']):
    # z = list(normalSIP['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
    datetime.datetime.strptime(i,'%b %d, %Y').year+2,datetime.datetime.strptime(i,'%b %d, %Y').year+3,
    datetime.datetime.strptime(i,'%b %d, %Y').year+4) in l.keys():
        # l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
        # datetime.datetime.strptime(i,'%b %d, %Y').year+2,datetime.datetime.strptime(i,'%b %d, %Y').year+3,datetime.datetime.strptime(i,'%b %d, %Y').year+4].append(None)
        continue
    else:
        l[datetime.datetime.strptime(i,'%b %d, %Y').year,datetime.datetime.strptime(i,'%b %d, %Y').year+1,
        datetime.datetime.strptime(i,'%b %d, %Y').year+2,datetime.datetime.strptime(i,'%b %d, %Y').year+3,
        datetime.datetime.strptime(i,'%b %d, %Y').year+4] = []
# print(l)
for z in l.keys():
    for date in list(normalSIP['Date']):
        if datetime.datetime.strptime(date,'%b %d, %Y').year in z:
            l[z].append(date)
        continue

# print(l)

my_columns = ['Year','XIRR']
normalSIP_fiveyear = pd.DataFrame(columns=my_columns)
for g in l.keys():
    # print(l[g])
    d=[]
    cf = []
    qty = 0
    if 2022 in g:
        continue
    for y in l[g]:
        z = list(normalSIP['Date'].loc[lambda x:x==y].index)
        if y == l[g][-1]:
            cf.append(-normalSIP.loc[z[0],'Investment Amount'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            qty += normalSIP.loc[z[0],'Qty']
            cf.append(qty*normalSIP.loc[z[0],'Buy Price'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
        else:
            cf.append(-normalSIP.loc[z[0],'Investment Amount'])
            d.append(datetime.datetime.strptime(y,'%b %d, %Y'))
            qty += normalSIP.loc[z[0],'Qty']
        
    # print(g,irr(d,cf))
    # print(d,cf)
    normalSIP_fiveyear = normalSIP_fiveyear.append(pd.Series([(f'{g[0]} - {g[-1]}'),irr(d,cf)],index=my_columns),ignore_index=True)

normalSIP_fiveyear

            

        


,Year,XIRR
0,2002 - 2006,0.3960
1,2003 - 2007,0.4433
2,2004 - 2008,-0.0144
3,2005 - 2009,0.1683
4,2006 - 2010,0.1510
5,2007 - 2011,0.0353
6,2008 - 2012,0.0877
7,2009 - 2013,0.0873
8,2010 - 2014,0.1638
9,2011 - 2015,0.1005


Daily SIP.csv Import and gettind dates and cashflow for IRR

In [16]:
dailySIP = pd.read_csv('SIP Details/Day SIP.csv')
dailySIP.drop(dailySIP.columns[[0]],axis=1,inplace=True)
d = []
cf = []
for i in dailySIP['Date']:
    if i == list(dailySIP['Date'])[-1]:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
    else:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
for x in dailySIP['Investment Amount']:
    if x == list(dailySIP['Investment Amount'])[-1]:
        cf.append(-x)
        qtytotal = dailySIP['Qty'].sum()
        ltp = list(dailySIP['Buy Price'])[-1]
        cf.append(qtytotal*ltp)
    else:
        cf.append(-x)
print('IRR')
print(irr(d,cf)*100,'%')


print(xirr(d,cf))
# print(d)

IRR
14.620000000000022 %
0.14613060099072772


Weekly SIP.csv Import and gettind dates and cashflow for IRR

In [17]:
weekSIP = pd.read_csv('SIP Details/Week SIP.csv')
weekSIP.drop(weekSIP.columns[[0]],axis=1,inplace=True)
d = []
cf = []
for i in weekSIP['Date']:
    if i == list(weekSIP['Date'])[-1]:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
    else:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
for x in weekSIP['Investment Amount']:
    if x == list(weekSIP['Investment Amount'])[-1]:
        cf.append(-x)
        qtytotal = weekSIP['Qty'].sum()
        ltp = list(weekSIP['Buy Price'])[-1]
        cf.append(qtytotal*ltp)
    else:
        cf.append(-x)
print('IRR')
print(irr(d,cf)*100,'%')


print(xirr(d,cf))
# print(d)

IRR
14.570000000000027 %
0.14565983065403998


Expiry SIP.csv Import and gettind dates and cashflow for IRR

In [18]:
expirySIP = pd.read_csv('SIP Details/Expiry SIP.csv')
expirySIP.drop(expirySIP.columns[[0]],axis=1,inplace=True)
d = []
cf = []
for i in expirySIP['Date']:
    if i == list(expirySIP['Date'])[-1]:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
    else:
        d.append(datetime.datetime.strptime(i,'%b %d, %Y'))
for x in expirySIP['Investment Amount']:
    if x == list(expirySIP['Investment Amount'])[-1]:
        cf.append(-x)
        qtytotal = expirySIP['Qty'].sum()
        ltp = list(expirySIP['Buy Price'])[-1]
        cf.append(qtytotal*ltp)
    else:
        cf.append(-x)
print('IRR')
print(irr(d,cf)*100,'%')

print(xirr(d,cf))
# print(d)

IRR
14.070000000000082 %
0.14067645142786245
